In [ ]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
# Data visualization
import numpy as np
import pandas as pd

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [ ]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")

    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''

    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


df = describe_dataset("./train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

In [ ]:
# load dataset
df = pd.read_csv("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "H", "label"] = 1
df.loc[df["label"] == "L", "label"] = 2

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 28520 
Number of columns: 69

Labels: 
label
0    9904
2    9546
1    9070
Name: count, dtype: int64



In [ ]:
# Standard Scaling of features
# Dump input scaler
with open("./input_scaler.pkl", "rb") as f2:
    sc = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [ ]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [ ]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [ ]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [ ]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)
tuner_3l.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 19s]
accuracy: 0.9954417943954468

Best accuracy So Far: 0.9992110729217529
Total elapsed time: 00h 03m 42s


In [ ]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x7dc38afde710>
Layer-2: 32 units, func:  <function tanh at 0x7dc333f7a680>
Layer-3: 3 units, func:  <function softmax at 0x7dc333f7a290>

Other params:
learning_rate: 0.001
Epoch 1/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9605 - loss: 0.1337 - val_accuracy: 0.9991 - val_loss: 0.0038
Epoch 2/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9976 - loss: 0.0091 - val_accuracy: 0.9993 - val_loss: 0.0032
Epoch 3/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9980 - loss: 0.0079 - val_accuracy: 0.9974 - val_loss: 0.0073
Epoch 4/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9983 - loss: 0.0072 - val_accuracy: 0.9919 - val_loss: 0.0244
Epoch 5/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9980 - loss: 0.0065 - val_accuracy: 0.9991 - val_loss: 0.0043
Epoch 6/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9983 - loss: 0.0066 - val_acc

In [ ]:
final_models["3_layers"] = model_3l

In [ ]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])

    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.998, 1.0, 0.999]","[0.999, 1.0, 0.998]","[0.999, 1.0, 0.999]","[[1913, 0, 2], [0, 1844, 0], [3, 0, 1942]]"


In [ ]:
# load dataset
test_df = pd.read_csv("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "H", "label"] = 1
test_df.loc[test_df["label"] == "L", "label"] = 2

print(f'Number of rows: {test_df.shape[0]} \nNumber of columns: {test_df.shape[1]}\n')
print(f"Labels: \n{test_df['label'].value_counts()}\n")

Number of rows: 710 
Number of columns: 69

Labels: 
label
1    241
2    235
0    234
Name: count, dtype: int64



In [ ]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [ ]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])

    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.986, 0.923, 1.0]","[0.915, 0.996, 0.991]","[0.949, 0.958, 0.996]","[[214, 20, 0], [1, 240, 0], [2, 0, 233]]"


In [ ]:

# Menyimpan model dalam format HDF5
model_3l.save('_model_plank_.h5')



In [ ]:
import tensorflow as tf

# Load your H5 model
model = tf.keras.models.load_model('_model_plank_.h5')  # Perbaiki nama file di sini

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('plank_detector.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpjybjz3ag'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 68), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  138276887256528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138276926674320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138276888050176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138276862690752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138276862589616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138276862576768: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [ ]:
import tensorflow as tf
from mediapipe.tasks.python.metadata.metadata_writers import model_bundle_writer

# Load your TFLite model
model_buffer = None
with open("plank_detector.tflite", 'rb') as f:
    model_buffer = f.read()

# Create model bundle writer
writer = model_bundle_writer.ModelBundleWriter(model_buffer)

# Add metadata
writer.add_metadata(
    subgraph_index=0,  # Main model subgraph
    input_metadata={
        "input": {
            "name": "pose_landmarks",
            "description": "Pose landmarks input",
            "content_type": "float32",
            "min_values": [-1.0],
            "max_values": [1.0],
        }
    },
    output_metadata={
        "output": {
            "name": "plank_classification",
            "description": "Plank pose classification",
            "content_type": "float32",
            "labels": ["Correct", "High Back", "Low Back"],
        }
    }
)

# Write the model bundle
bundle_buffer = writer.write()

# Save as .task file
with open('plank_detector.task', 'wb') as f:
    f.write(bundle_buffer)

ImportError: cannot import name 'model_bundle_writer' from 'mediapipe.tasks.python.metadata.metadata_writers' (/usr/local/lib/python3.10/dist-packages/mediapipe/tasks/python/metadata/metadata_writers/__init__.py)